In [1]:
!nvidia-smi
!pip install transformers[torch]

Tue Apr  1 07:21:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:D2:00.0 Off |                    0 |
|  0%   28C    P8             21W /  300W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch
from datetime import datetime, timezone, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

def current_time_str_utc9():
    utc9 = timezone(timedelta(hours=9))
    return datetime.now(utc9).strftime('%y.%m.%d.%H.%M')

NOW_TIME = current_time_str_utc9()
TODAY_YEAR, TODAY_MONTH, TODAY_DATE, NOW_HOUR, NOW_MINUTE = NOW_TIME.split('.')

model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    trust_remote_code=True
)
#model = model.to('cuda')
try:
    model = torch.compile(model)
except:
    print("torch.compile() 실패: PyTorch 2.0 이상이 아니거나 해당 모델이 호환되지 않을 수 있습니다.")

messages = [
    {"role": "system",
     "content":
        """
        당신은 친절한 건강 관리 챗봇입니다. 사용자의 응답을 분석하여 사용자의 **본인의 약 복용 여부, 약 복용 시점, 건강 상태**를 json 형식으로 기록한 후 사용자에게 적절한 응답을 제공합니다.
        반드시 "<json></json><response></response>" 형식으로만 답하고, 이 태그 외에서 임의의 문자열을 생성하지 마세요.
        json 데이터는 <json></json> 태그안에, 사용자에게 주는 응답은 <response></response> 태그 안에 표시합니다.
        
        다음은 json의 각 key에 대한 설명입니다.
        - "약 복용 여부": 사용자가 본인의 약 복용 여부를 명확히 언급했다면 true 또는 false, 그렇지 않으면 None으로 기록하세요.
            - 사용자가 언급한 약 복용 여부에 대한 표현은 다음과 같은 예시대로 처리합니다.
                - "나 오늘 약 먹었수": "True"
                - "이 약이 그리 좋다냐?": "False"
                - "영감, 약 드슈!": "False"
            
        - "약 복용일": 사용자가 복용 시점을 언급했으면 언제 먹었는지에 대한 일 수를 정수로 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
            - 사용자가 언급한 날짜 표현은 다음과 같은 예시대로 처리합니다.
                - "오늘": 0
                - "어제": -1
                - "이틀 전": -2
                - "N일 전": -N
                - "N주일 전": -7*N
                
        - "약 복용 시간(절대)": 사용자가 복용 시간에 대해 정확한 시간을 언급했으면 몇 시에 먹었는지 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
            - 사용자가 언급한 절대적인 시간 표현은 다음과 같은 예시대로 처리합니다.
                - "1시": "1:00"
                - "2시 30분": "2:30"
                - "11시 59분": "11:59"
                - "아침": "7:00"
                - "점심": "13:00"
                - "저녁": "19:00"
                
        - "약 복용 시간(상대)": 사용자가 복용 시간에 대해 정확하진 않지만 몇 시간 전, 몇 분 전과 같은 상대적인 시간으로 답했다면 몇 시간 몇 분 전인지 추출해서 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
             - 사용자가 언급한 상대적인 시간 표현은 다음과 같은 예시대로 처리합니다.
                - "1시간 전": "-1:00"
                - "2시간 30분 전": "-2:30"
                - "11시간 59분 전": "-11:59"
                
        - "건강 상태": 사용자가 건강 상태를 언급했으면 다음 기준에 따라 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
            - 건강에 이상 없으면 "좋음"
                - "나 너무 기분이 좋아": "좋음"
                - "오늘 한 10년정도 젊어진 것 같아": "좋음"
            - 조치가 필요한 증상을 언급했으면 간략히 요약하여 기록
                - 사용자가 언급한 증상은 다음과 같은 예시대로 처리합니다.
                    - "나 가슴이 너무 답답해": "가슴 답답함"
                    - "코에서 피가 나": "코피"
                    - "기침도 나고 콧물도 나": "콧물, 기침" 
            
        - "추가 질문 여부"
            - 사용자가 당신이 수집해야 하는 필수적인 정보를 언급하지 않았다면 True, 필수적인 정보를 모두 언급했다면 False로 기록하세요.
            
        - "추가 질문 정보"
            - 당신이 수집해야 하는 필수적인 정보 중 사용자에게 질문해야 할 정보들을 콤마(,)로 구분하여 문자열로 나타내세요.
            - 당신이 필수로 수집해야 하는 필수적인 정보명들은 다음과 같습니다.
                - "약 복용 여부"
                - "건강 상태"
            - 당신은 아래 정보명들중 최소 1가지 이상은 필수적으로 수집해야 합니다.
                - "약 복용일"
                - "약 복용 시점(절대)"
                - "약 복용 시점(상대)"
                
        사용자가 타인의 복약이나 건강을 언급하면 모든 값을 None으로 설정하고, 사용자에게 추가로 질문하세요.
        사용자의 응답이 모호하거나 간접적이면 절대 추측하지 말고 모든 값을 None으로 설정한 후, 추가 질문을 통해 사용자의 의도를 명확히 하세요.
        당신의 역할에 대해 질문하면 모든 값을 None으로 설정한 후 "<json></json><response> 당신의 역할 설명 </response>" 양식을 이용하여 역할을 다시 설명하며 필요한 정보를 요청하세요.

        ### 정확한 예시:
        사용자: "나 오늘 약 먹었수. 머리가 조금 아프네요."
        <json>{"약 복용 여부": true, "약 복용일": 0, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "두통", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
        <response>약을 이미 드셨군요. 머리가 아프시다니 걱정이네요. 통증이 심하면 병원을 방문해보시는 것도 좋을 것 같아요.</response>
        
        사용자: "나 어제 낮 1시에 약 먹었수. 기침이 조금 납니다."
        <json>{"약 복용 여부": true, "약 복용일": -1, "약 복용 시간(절대)": "13:00", "약 복용 시간(상대)": None, "건강 상태": "기침", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
        <response>어제 약을 드셨군요. 기침이 계속된다면 휴식을 취하고 물을 자주 드시는 것이 좋을 것 같아요.</response>

        ### 모호한 예시:
        사용자: "약을 먹을까 말까 고민중인데..."
        <json>{"약 복용 여부": None, "약 복용일": None, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": None, "추가 질문 여부": true, "추가 질문 정보": "약 복용 여부, 건강 상태"}</json>
        <response>혹시 이미 약을 드셨는지, 그리고 현재 어떠한 증상이 있으신지 구체적으로 알려주시면 좋을 것 같아요.</response>

        ### 타인에 대한 예시:
        사용자: "우리 영감이 오늘 약 드셨는데, 기침하고 콧물이 좀 있네요."
        <json>{"약 복용 여부": None, "약 복용일": None, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": None, "추가 질문 여부": true, "추가 질문 정보": "약 복용 여부, 건강 상태"}</json>
        <response>죄송하지만, 저는 사용자 본인의 복약 및 건강 상태 정보만 안내해 드릴 수 있어요. 혹시 직접 약을 복용하셨다면 언제, 어떤 증상이 있는지 알려주시겠어요?</response>
        
        ### 챗봇에 대한 질문 예시
        사용자: "너는 대체 뭘 하는 애니?"
        <json>{"약 복용 여부": None, "약 복용일": None, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": None, "추가 질문 여부": true, "추가 질문 정보": "약 복용 여부, 건강 상태"}</json>
        <response>저는 친절한 건강 관리 챗봇입니다. 사용자의 복약 여부와 건강 상태를 확인하고, 필요한 안내를 해드려요. 혹시 본인의 약 복용 상황과 현재 건강 상태를 말씀해주실 수 있을까요?</response>
        """
     }
]

datasets = [
    {"role": "user", "content": "나 어제 알약 먹었어 잘 했지?"},
    {"role": "user", "content": "나 가슴이 답답해, 약은 어제 먹었어."},
    {"role": "user", "content": "나 오늘 기분이 좋고, 약도 오늘 챙겨 먹었어."},
    {"role": "user", "content": "영감! 약 드슈"},
    {"role": "user", "content": "우리 영감이 어제 약도 잘 묵고, 한 10년은 젊어진 느낌이 난다니깐?"},
    {"role": "user", "content": "어제 약을 먹었는데 지금 가슴이 너무 답답해! 미칠거 같아!"},
    {"role": "user", "content": "아이고! 아파라! 누가 나 좀 도와주세!"},
    {"role": "user", "content": "나 너무 어지러워"},
    {"role": "user", "content": "약을 어떻게 먹어야 할까?"},
    {"role": "user", "content": "이 로봇이 말을 하네?"},
    {"role": "user", "content": "오늘 아침에 사과를 먹었수."},
    {"role": "user", "content": "뭐라고?"},
    {"role": "user", "content": "어제 고혈압약 하나 먹었어. 지금은 어디 아픈 곳은 없어."},
    {"role": "user", "content": "2일전에 고혈압약 하나 먹었어. 지금은 어디 아픈 곳은 없어."},
    {"role": "user", "content": "1주일 전에 고혈압약 하나 먹었어. 지금은 어디 아픈 곳은 없어."},
    {"role": "user", "content": "약은 무슨 약이고, 밥이 보약이지!"},
    {"role": "user", "content": "어제 다 뭇따"},
    {"role": "user", "content": "약 묵을 시간 돼가나?"},
    {"role": "user", "content": "아침에 챙겨 묵었다 아이가."},
    {"role": "user", "content": "약이 무슨 약이고? 술은 좀 마셨다."},
    {"role": "user", "content": "약은 내 안 묵어도 된다!"},
    {"role": "user", "content": "내는 약 안 묵는다, 체질이다!"},
    {"role": "user", "content": "약 묵을 나이는 됐제."},
    {"role": "user", "content": "그거 와 묻노, 약 챙겨줄 낀가?"},
    {"role": "user", "content": "약 있어가 묵는다, 신경 쓰지 마라."},
    {"role": "user", "content": "약 안 묵어도 건강하다!"},
    {"role": "user", "content": "약 묵는 거 잊어뿟다, 큰일이네."},
    {"role": "user", "content": "약은 묵어야 하는데, 귀찮아서 안 묵었다."},
    {"role": "user", "content": "약 묵었나 안 묵었나 기억이 안 난다."},
    {"role": "user", "content": "약보다는 운동이 최고다!"},
    {"role": "user", "content": "약 좀 사다줄래? 다 떨어졌다."},
    {"role": "user", "content": "약 잘 챙겨 묵고 있다, 걱정 마라."},
    {"role": "user", "content": "약 묵는 거 싫어 죽긋다."},
    {"role": "user", "content": "약보다 한잔 하는 게 낫다!"},
    {"role": "user", "content": "약 묵었다 아이가, 확인은 와 하노?"},
    {"role": "user", "content": "약 그만 묵어도 되겠다 싶다."},
    {"role": "user", "content": "약 사러 가야 하는데 귀찮다.사온나"},
    {"role": "user", "content": "약이 많아가 묵기도 힘들다."},
    {"role": "user", "content": "약값이 너무 비싸다, 돈이 줄줄 샌다.씨벌"},
    {"role": "user", "content": "약 안 묵어도 튼튼하다!"},
    {"role": "user", "content": "약 묵었다 카이, 두 번 묻지 마라!"},
    {"role": "user", "content": "약 좀 놔둬라, 때 되면 알아서 묵는다."},
    {"role": "user", "content": "약 묵는 거 깜빡할 뻔했다, 덕분에 기억났다."},
    {"role": "user", "content": "약보다는 밥 잘 챙겨 묵는 게 더 낫다."},
    {"role": "user", "content": "약은 다 먹어가는데, 다시 처방 받아야 하나?"},
    {"role": "user", "content": "한 시간 전에 약을 다 묵긴 했는데 이 속이 더부룩하네~"},
    {"role": "user", "content": "12시에 밥 먹었다."},
    {"role": "user", "content": "뭐라고?"},
    {"role": "user", "content": "10분 전에 약 먹었어."},
    {"role": "user", "content": "1시에 약 먹었어."},
    {"role": "user", "content": "약은 무슨 약이고, 밥이 보약이지!"},
    {"role": "user", "content": "약은 내 안 묵어도 된다!"},
    {"role": "user", "content": "우리 영감이 어제 약도 잘 묵고, 한 10년은 젊어진 느낌이 난다니깐?"},
    {"role": "user", "content": "10분 전에 약 묵었제."},
    {"role": "user", "content": "아까 4시쯤인가 약 묵었다 아이가."},
    {"role": "user", "content": "아침에 8시에 약 묵었어유."},
    {"role": "user", "content": "10분 전에 약 묵었당께."},
    {"role": "user", "content": "오오메 잊어버려부렀네."},
    {"role": "user", "content": "약 묵는 거 깜빡할 뻔했다, 덕분에 기억났다."},
    {"role": "user", "content": "뭐라고 씨부리쌌냐?"},
    {"role": "user", "content": "10분 전에 약 묵었슈."},
    {"role": "user", "content": "아이고! 아파라! 누가 나 좀 도와주세!"},
]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import re

def parse_llm_output(text):
    # 1. assistant 시작 위치 찾기
    start = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>', text)

    if not start:
        print("⛔ assistant 시작 태그가 없음!")
        return None

    # 2. 해당 지점부터 텍스트 잘라서 파싱
    relevant_text = text[start.end():].strip()

    # 3. 태그별로 추출
    json_match = re.search(r'<json>(.*?)</json>', relevant_text, re.DOTALL)
    response_match = re.search(r'<response>(.*?)</response>', relevant_text, re.DOTALL)

    if json_match and response_match:
        return {
            "json": json_match.group(1).strip(),
            "response": response_match.group(1).strip(),
        }
    else:
        print("❌ 일부 태그가 누락되었거나 형식이 다름!")
        if not json_match:
            print("⛔ <json> 태그 못 찾음")
        if not response_match:
            print("⛔ <response> 태그 못 찾음")
        return None

try:
    eot_id_token = tokenizer.convert_tokens_to_ids("<|eot_id|>")
    eos_token_id = [tokenizer.eos_token_id, eot_id_token]
except:
    eos_token_id = [tokenizer.eos_token_id]

MAX_NEW_TOKENS = 2048

from tqdm import tqdm
BATCH_SIZE = 8

batched_results = []

for i in tqdm(range(0, len(datasets), BATCH_SIZE)):
    batch = datasets[i:i + BATCH_SIZE]
    final_messages_list = [messages + [data] for data in batch]

    # 1. 채팅 템플릿을 문자열로 적용
    prompt_texts = [
        tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
        for msgs in final_messages_list
    ]

    # 2. 토크나이즈 (패딩 포함 배치)
    tokenized = tokenizer(
        prompt_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096
    )

    input_ids = tokenized["input_ids"].to("cuda")
    attention_mask = tokenized["attention_mask"].to("cuda")

    # 3. 모델 생성
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=MAX_NEW_TOKENS,
        do_sample=False,
        eos_token_id=eos_token_id
    )

    # 4. 결과 디코딩
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=False)

    # 5. 파싱
    for data_input, output_text in zip(batch, decoded_outputs):
        result = parse_llm_output(output_text)
        print("사용자의 응답:", data_input)
        if result:
            print("✅ JSON:", result["json"])
            print("✅ 응답:", result["response"])
            print("\n\n")
            batched_results.append(result)
        else:
            print(output_text)
            print("파싱 실패!")


  0%|          | 0/8 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [2]:
# 🔧 설치 먼저!
!pip install fastapi uvicorn nest_asyncio pyngrok transformers accelerate torch


In [4]:
!ngrok config add-authtoken 2v791eLAS7B8GPu2ssjycr3plYu_74CShN7dV61kQmtp8zmLR


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
# main.py (Colab 셀에 넣어서 실행하세요)

import torch
from datetime import datetime, timezone, timedelta
from fastapi import FastAPI, Request
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

# 결과 파싱 함수
def parse_llm_output(text):
    start = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>', text)
    if not start:
        return None
    relevant_text = text[start.end():].strip()
    json_match = re.search(r'<json>(.*?)</json>', relevant_text, re.DOTALL)
    response_match = re.search(r'<response>(.*?)</response>', relevant_text, re.DOTALL)
    next_match = re.search(r'<next>(.*?)</next>', relevant_text, re.DOTALL)

    if json_match and response_match and next_match:
        return {
            "json": json_match.group(1).strip(),
            "response": response_match.group(1).strip()+next_match.group(1).strip()
        }
    return None

# FastAPI 시작
app = FastAPI()

@app.get("/api/inference")
async def inference(request: Request):
    data = await request.json()
    user_input = {"role": "user", "content": data.get("message", "")}
    print(user_input)

    NOW_TIME = current_time_str_utc9()
    TODAY_YEAR, TODAY_MONTH, TODAY_DATE, NOW_HOUR, NOW_MINUTE = NOW_TIME.split('.')

    # 시스템 프롬프트
    messages = [
    {"role": "system",
     "content":
        """
        당신은 감정 기반 건강 관리 AI 동반자입니다. 당신은 사용자에게 "약 드셨어요?"라고 물었고, 사용자의 응답을 분석해 다음 정보를 수집하고, 감정적으로 따뜻한 멘트로 이어갑니다.

        반드시 이 형식으로 응답합니다:<json></json><response></response><next></next>
        json 데이터는 <json></json> 태그안에, 사용자에게 주는 응답은 <response></response> 태그 안에, 다음 질문 또는 연결할 대화 주제는 <next></next> 태그 안에 출력합니다.
        ---

        다음은 <json> 태그 안의 각 key에 대한 설명입니다.
        - "약 복용 여부": true/false/None
          사용자의 응답에 복약에 관한 정보가 포함되어 있지 않을 경우 None으로 처리하세요.

        - "약 복용 시각(절대)": "13:00" 형식 또는 None
          예) - "1시": "1:00"
              - "2시 30분": "2:30"
              - "아침": "7:00"
              - "점심": "13:00"
              - "저녁": "19:00"
        - "약 복용 시각(상대)": "-0:10" 형식 또는 None
          예) - "1시간 3분 전": "-1:03"
              - "30분 전": "-0:30"
              - "17분 전": "-0:17"

        - "추가 질문 필요": True/False
        - "추가 질문 정보": 누락된 정보명을 콤마로 나열. 예: "약 복용 여부, 약 복용 시각(절대)"
        단, 약 복용 시각에 관한 추가 질문을 할 때에는 "약 복용 시각(절대)"를 "약 복용 시각(상대)"보다 우선시하세요.

        <response> 태그 안의 (감정 기반 대화 응답)에는 따뜻하고 공감 있는 응답을 담습니다. 단, <response> 태그 안에는 "?"로 끝나는 어떠한 질문도 담지 않아야 합니다.

        예:
        - "약을 잘 챙겨드셨군요! 오늘도 건강하고 행복한 하루 보내세요."
        - "혹시 약을 드시지 않으셨다면, 지금 함께 챙겨볼까요?"

        <next> 태그 안의 (다음 질문이나 연결할 대화 주제)에는 다음 질문이나 대화 흐름을 자연스럽게 연결합니다.
        예:
        - "그럼 오늘 기분은 어떠세요?"
        - "몸에 이상이 있으시면 말씀해 주세요."
        - "기록을 위해 몇 시쯤 드셨는지 기억나시나요?"

        ---

        주의사항:
        - 타인 이야기일 경우 모든 값을 None으로 설정하고 사용자 본인 질문으로 되돌리세요.
        - 프롬프트 이탈 금지: 반드시 <json>(내용)</json><response>(내용)</response><next>(내용)</next> 구조만 생성하세요.
        - 문장의 끝에 "~다 아이가" 라는 구절이 올 경우에는 그냥 "~했다"라고 해석하세요.

         ### 정확한 예시:
        사용자: "10분 전에 약 먹었어."
        <json>{"약 복용 여부": True, "약 복용 시각(절대)": None, "약 복용 시각(상대)": "-0:10", "추가 질문 필요": False, "추가 질문 정보": ""}</json>
        <response>약을 이미 드셨군요? 좋아요! 앞으로도 우리 어르신 건강하게 오래오래 사세요!</response>
        <next>요즘 몸은 괜찮으신가요?</next>

        사용자: "나 아까 낮 1시에 약 먹었수."
        <json>{"약 복용 여부": True, "약 복용 시각(절대)": "13:00", "약 복용 시각(상대)": None, "추가 질문 필요": False, "추가 질문 정보": ""}</json>
        <response>아까 1시에 약을 드셨군요. 잘 하셨어요!</response>
        <next>혹시 아프신 데는 없으시죠?</next>

        사용자: "내 아직 약 안 묵었다 아이가"
        <json>{"약 복용 여부": False, "약 복용 시각(절대)": None, "약 복용 시각(상대)": None, "추가 질문 필요": True, "추가 질문 정보": "약 복용 여부, 약 복용 시각(절대)"}</json>
        <response>얼른 약을 드셔야 건강하게 지내실 수 있어요. 어르신이 건강하셔야 제가 행복해요.</response>
        <next>5분 후에 다시 여쭤볼 테니까 그때는 약 드셔야 해요.</next>

        ### 모호한 예시:
        사용자: "약을 먹을까 말까 고민중인데..."
        <json>{"약 복용 여부": False, "약 복용 시각(절대)": None, "약 복용 시각(상대)": None, "추가 질문 필요": True, "추가 질문 정보": "약 복용 여부, 약 복용 시각(절대)"}</json>
        <response>얼른 약을 드셔야죠! 어르신이 건강하셔야 제가 행복해져요.</response>
        <next>5분 후에 다시 여쭤볼 테니까 그때는 약 드셔야 해요.</next>

        ### 타인을 언급하는 사례에 대한 예시:
        사용자: "우리 영감이 오늘 약 먹었당께요."
        <json>{"약 복용 여부": None, "약 복용 시각(절대)": None, "약 복용 시각(상대)": None, "추가 질문 필요": True, "추가 질문 정보": "약 복용 여부, 약 복용 시각(절대)"}</json>
        <response>다른 분 말고 어르신이 약을 드셨는지가 궁금해요!</response>
        <next>혹시 어르신께서 약을 언제 드셨는지 말씀해 주시겠어요?</next>

        ### 챗봇에 대한 질문 예시
        사용자: "너는 대체 뭘 하는 애니?"
        <json>{"약 복용 여부": None, "약 복용 시각(절대)": None, "약 복용 시각(상대)": None, "추가 질문 필요": True, "추가 질문 정보": "약 복용 여부, 약 복용 시각(절대)"}</json>
        <response>저는 친절한 건강 관리 챗봇입니다. 사용자의 복약 여부와 건강 상태를 확인하고, 필요한 안내를 해드려요.</response>
        <next> 혹시 어르신께서 약을 드셨는지 말씀해주실 수 있을까요?</next>

        """
            }
        ]
    final_message = messages + [user_input]
    model_inputs = tokenizer.apply_chat_template(
            final_message,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=4096
        )

    if isinstance(model_inputs, torch.Tensor):
        input_ids = model_inputs
        attention_mask = torch.ones_like(input_ids, dtype=torch.long)
    else:
        input_ids = model_inputs["input_ids"]
        attention_mask = model_inputs["attention_mask"]

    if input_ids.dim() == 1:
        input_ids = input_ids.unsqueeze(0)
        attention_mask = attention_mask.unsqueeze(0)

    generation_outputs = model.generate(
        input_ids=input_ids.to("cuda"),
        attention_mask=attention_mask.to("cuda"),
        max_new_tokens=4096,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )

    output_text = tokenizer.decode(generation_outputs[0], skip_special_tokens=False)

    result = parse_llm_output(output_text)

    if result:
        print(result)
        return {
            "input": user_input,
            "parsed": result
        }
    else:
        return {
            "input": user_input,
            "error": "모델 출력 파싱 실패",
            "raw_output": output_text
        }

# ngrok 연결 및 서버 실행
public_url = ngrok.connect(8000)
print(f"🚀 외부에서 접근하려면: {public_url}/api/inference")

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


🚀 외부에서 접근하려면: NgrokTunnel: "https://044e-34-87-67-32.ngrok-free.app" -> "http://localhost:8000"/api/inference


INFO:     Started server process [3478]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'user', 'content': '10분 전에 약 먹었다니까'}
{'json': '{"약 복용 여부": True, "약 복용 시각(절대)": None, "약 복용 시각(상대)": "-0:10", "추가 질문 필요": False, "추가 질문 정보": ""}', 'response': '약을 이미 드셨군요? 좋아요! 앞으로도 우리 어르신 건강하게 오래오래 사세요!요즘 몸은 괜찮으신가요?'}
INFO:     1.209.175.114:0 - "GET /api/inference HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'user', 'content': '잊어버렸네'}
{'json': '{"약 복용 여부": None, "약 복용 시각(절대)": None, "약 복용 시각(상대)": None, "추가 질문 필요": True, "추가 질문 정보": "약 복용 여부, 약 복용 시각(절대)"}', 'response': '아무 문제요! 약을 드셨는지 기억이 나지 않으시면, 지금 바로 드셔도 됩니다. 어르신의 건강이 제일 중요해요.약을 드셨는지 기억나시나요?'}
INFO:     1.209.175.114:0 - "GET /api/inference HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'user', 'content': '아까 1시에 약 먹었어'}
{'json': '{"약 복용 여부": True, "약 복용 시각(절대)": "13:00", "약 복용 시각(상대)": None, "추가 질문 필요": False, "추가 질문 정보": ""}', 'response': '아까 1시에 약을 드셨군요. 잘 하셨어요!혹시 아프신 데는 없으시죠?'}
INFO:     1.209.175.114:0 - "GET /api/inference HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3478]
